# ISHEEP WGS DATASET

This is an attempt to collect SNPs from [isheep WGS dataset](https://ngdc.cncb.ac.cn/isheep/download) and track them in to smarter database. There absolutely no info regarding *probeset IDs*, so the only way to collect SNPs from this dataset relies on position on *OAR4*. Data is divided by chromosomes and where downloaded from [isheep WGS ftp folder](ftp://download.big.ac.cn/isheep/SNP).

## Data preparation

Files were compressed using the standard `gzip` utility. Unpack all files and compress them using `bgzip`, then index with `tabix`:

```bash
for compressed in $(ls *.vcf.gz); do echo "Processing " $compressed; vcf="${compressed%.*}"; bgzip -d --stdout $compressed | bgzip -@24 --compress-level 9 --stdout > $vcf.bgzip ; done
for compressed in $(ls *.bgzip); do echo "Processing " $compressed; bgzip --test $compressed ; done
for compressed in $(ls *.bgzip); do echo "Processing " $compressed; vcf="${compressed%.*}.gz" ; mv $compressed $vcf ; tabix $vcf ; done
```

Now I can try to extract my probes relying on positions and `tabix`. The point is that I could have more probe on the same position, so I can't assign a unique `VariantSheep.name` to a certain SNP; Moreover positions need to be checked against `OAR4` before querying VCF. For the moment, I will try to extract the information I need relying on the data I have

In [1]:
import csv
import logging

from tqdm.notebook import tqdm
from mongoengine.errors import DoesNotExist, MultipleObjectsReturned

from src.features.smarterdb import global_connection, VariantSheep
from src.features.utils import get_interim_dir, get_project_dir
from src.features.plinkio import BinaryPlinkIO, CodingException
from src.data.common import WORKING_ASSEMBLIES

In [2]:
_ = global_connection()
OAR4 = WORKING_ASSEMBLIES["OAR4"]
OAR3 = WORKING_ASSEMBLIES["OAR3"]
logger = logging.getLogger('src.features.plinkio')
logger.setLevel(logging.WARNING)

## Creating the regions file

As described by the `tabix` documentation, I can collect samples by region. Chromosome and position in a *tab separated* file is enough. I have a VCF for each chromosome, so I need to collect data by chromosomes. I could have the same position more times in this data file (since different probeset could be present more times): i need to extract the same record more than once

In [3]:
chromosomes = [str(chrom) for chrom in range(1, 27)] + ["X"]

for chrom in tqdm(chromosomes):
    regions_filename = get_interim_dir() / f"chr{chrom}_regions.tsv"
    
    if regions_filename.exists():
        # save some time, skip evaluation
        continue
        
    condition = OAR4._asdict()
    condition['chrom'] = chrom
    variants = VariantSheep.objects.filter(
        locations__match=condition
    ).fields(
        elemMatch__locations=OAR4._asdict(),
        name=1,
        rs_id=1
    )
    
    locations = set()
    
    for variant in variants:
        location = (variant.locations[0].chrom, variant.locations[0].position)
        
        if location not in locations:
            locations.add(location)
            
    locations = sorted(list(locations), key=lambda location: location[1])
    
    with open(regions_filename, "w") as handle:
        writer = csv.writer(handle, delimiter="\t", lineterminator="\n")
        for location in locations:
            writer.writerow(location)
            

  0%|          | 0/27 [00:00<?, ?it/s]

## Extract variants from VCF

It's time to extract the sheep smarter variants from the VCF files. Then merge all files into one VCF:

```bash
for i in $(seq 1 26); do echo "tabix -h -R chr$i\_regions.tsv output_chr$i.snp.filtered.vcf.gz | bgzip --compress-level 9 --stdout > smarter_chr$i\_regions.vcf.gz; tabix smarter_chr$i\_regions.vcf.gz" >> commands.sh ; done
echo "tabix -h -R chrX\_regions.tsv output_chrX.snp.filtered.vcf.gz | bgzip --compress-level 9 --stdout > smarter_chrX\_regions.vcf.gz; tabix smarter_chrX\_regions.vcf.gz" >> commands.sh
parallel --jobs 12 < commands.sh
vcf-concat smarter_chr*.vcf.gz | bgzip -@12 --stdout > WGS-all/WGS-all.smarter.vcf.gz
tabix WGS-all/WGS-all.smarter.vcf.gz
plink --chr-set 26 --allow-extra-chr --vcf WGS-all/WGS-all.smarter.vcf.gz --make-bed --double-id --out WGS-all/WGS-all.smarter
```

Please note that in the final plink file `WGS-all.smarter.bim` SNPs have no name: this need to be fixed with the `VariantSheep.name` field in order to be merged with the SMARTER dataset. I can define a custom method to search variants by position (not by name)

In [4]:
class CustomBinaryPlinkIO(BinaryPlinkIO):
    def process_pedfile(self, coding="top"):
        for line in tqdm(self.read_pedfile(), total=len(self.plink_file.get_samples())):
            _ = self._process_genotypes(line, coding)
            
        return True
    
    def is_top(self):
        try:
            return self.process_pedfile(coding='top')
        
        except CodingException:
            return False
    
    def is_forward(self):
        try:
            return self.process_pedfile(coding='forward')
        
        except CodingException:
            return False
        
    def fetch_coordinates_by_positions(self, src_assembly: dict, dst_assembly: dict):
        # reset meta informations
        self.locations = list()
        self.filtered = set()
        self.variants_name = list()

        # helper function
        def skip_index(idx):
            # skip this variant (even in ped)
            self.filtered.add(idx)

            # need to add an empty value in locations (or my indexes
            # won't work properly). The same for variants name
            self.locations.append(None)
            self.variants_name.append(None)
        
        for idx, record in enumerate(tqdm(self.mapdata, mininterval=2)):
            location = src_assembly.copy()
            location["chrom"] = record.chrom
            location["position"] = record.position
            
            # attempt to fix-up 27 chrom
            if int(location["chrom"]) == 27:
                location["chrom"] = "X"
        
            try:
                variant = self.VariantSpecies.objects(
                    locations__match=location,
                ).fields(
                    elemMatch__locations=dst_assembly,
                    name=1,
                    rs_id=1
                ).get()
            
            except DoesNotExist as e:
                logger.warning(
                    f"Couldn't find {record.chrom}:{record.position} in {src_assembly}"
                    f" assembly: {e}")

                skip_index(idx)

                # don't check location for missing SNP
                continue
                
            except MultipleObjectsReturned as e:
                # tecnically, I could return the first item I found in my database
                # skip, for the moment
                logger.debug(
                    f"Got multiple {record.chrom}:{record.position} in {src_assembly}"
                    f" assembly: {e}")

                skip_index(idx)

                # don't check location for missing SNP
                continue
                
            # using projection I will have only one location if I could
            # find a SNP
            location = variant.locations[0]

            # track data for this location
            self.locations.append(location)

            # track variant.name read from database (useful when searching
            # using probeset_id)
            self.variants_name.append(variant.name)

In [5]:
prefix = str(get_project_dir() / "data/external/SHE/ISHEEP/WGS-all/WGS-all.smarter")
plinkio = CustomBinaryPlinkIO(prefix= prefix, species="Sheep", chip_name="IlluminaOvineSNP50")
plinkio.read_mapfile()
plinkio.fetch_coordinates_by_positions(src_assembly=OAR4._asdict(), dst_assembly=OAR3._asdict())

  0%|          | 0/602009 [00:00<?, ?it/s]

Is this file in illumina top?

In [6]:
plinkio.is_top()

  0%|          | 0/355 [00:00<?, ?it/s]

Error for SNP 2:.: T/T <> A/C


False

Ok, is this in forward coordinates (as supposed to be for a VCF?)

In [7]:
plinkio.is_forward()

  0%|          | 0/355 [00:00<?, ?it/s]

Error for SNP 59:.: T/T <> A/G


False

This is unexpected: why this genotypes can't be converted? Get some stuff

In [8]:
def get_info_snp(idx):
    sample_genotypes = next(plinkio.read_pedfile())

    snp = plinkio.mapdata[idx]
    a1 = sample_genotypes[6+idx*2]
    a2 = sample_genotypes[6+idx*2+1]
    genotype = [a1, a2]
    location = plinkio.locations[idx]
    name = plinkio.variants_name[idx]

    if not location.is_forward(genotype):
        print(f"SNP {idx}({genotype}):{name} is not in forward coordinates)")
    if not location.is_top(genotype):
        print(f"SNP {idx}({genotype}):{name} is not in top coordinates)")
        
get_info_snp(59)

SNP 59(['T', 'T']):DU281551_498.1 is not in forward coordinates)
SNP 59(['T', 'T']):DU281551_498.1 is not in top coordinates)


This SNP in particoular seems to have issue: I know from other sources that it should be C/T in forward, however is stored in A/G in snpchimp and I can't use any other sources to derive the correct genotypes. So let's drop this SNPs and similar SNPs from the dataset

In [9]:
def drop_variants(plinkio):
    counter = 0
    
    for idx, locus in enumerate(plinkio.plink_file.get_loci()):
        location = plinkio.locations[idx]
        
        if not location:
            # some SNPs have already been removed
            continue
        
        genotype = locus.allele1, locus.allele2
        
        # remove the non-forward variants
        if not location.is_forward(genotype):
            # this SNPs should be removed
            counter += 1
            
            # add SNP to filtered set
            plinkio.filtered.add(idx)

    
    print(f"{counter} SNPs are not in forward and were removed")
    
drop_variants(plinkio)

1504 SNPs are not in forward and were removed


Test for forward again (this time I suppose it will work without problems):

In [10]:
plinkio.is_forward()

  0%|          | 0/355 [00:00<?, ?it/s]

True

What about try allelic SNPs? consider 1:811144 which is C->T,G: only two alleles are considered. Sample *0624D_L2_321* which should be *G/G* will be set as missing:

In [11]:
def convert(genotype, locus):
    # in binary format, allele2 is REF allele1 ALT
    if genotype == 0:
        return locus.allele1, locus.allele1
    elif genotype == 1:
        return locus.allele2, locus.allele1
    elif genotype == 2:
        return locus.allele2, locus.allele2
    elif genotype == 3:
        return "0", "0"
    else:
        raise CodingException("Genotype %s Not supported" % genotype)

for locus, row in zip( plinkio.plink_file.get_loci(), plinkio.plink_file ):
    if locus.chromosome == "1" and locus.bp_position == 811144:
        for sample, genotype in zip( plinkio.plink_file.get_samples(), row ):
            if sample.iid == "0624D_L2_321":
                print(sample, convert(genotype, locus))
        break

0624D_L2_321 0624D_L2_321 -9 -9 ('0', '0')


Which genotypes will be chosen? I suppose the most frequent, but I need to ensure this (however, since I remove the non forward snp, those SNPs will be filtered out)